# Demand adjustment to Economic

In [ ]:
import pandas as pd
import geopandas as gpd
# Read the compressed CSV file into a DataFrame
energy_df = pd.read_csv('data/US_data/US_electricity/NREL/energy.csv.gzip', compression='gzip')
us_states = gpd.read_file('data/cb_2018_us_state_500k.shp')
us_nation = gpd.read_file('data/US_data/cb_2018_us_nation_5m.shp')
df_net_benefit_1=pd.read_csv('data/5.1_net_expected_benefit_results.csv')


In [32]:
energy_df['STATE'].unique()

array(['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA',
       'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA',
       'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA',
       'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND',
       'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI',
       'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE',
       'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA',
       'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA',
       'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE',
       'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON',
       'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'], dtype=object)

In [ ]:
# For each scenario, print the length of the filtered DataFrame using the same filter except for SCENARIO
for scenario in energy_df['SCENARIO'].unique():
    filtered_df = energy_df[
        (energy_df['STATE'] == 'CALIFORNIA') &
        (energy_df['YEAR'] == 2050) &
        (energy_df['SECTOR'] == 'RESIDENTIAL') &
        (energy_df['SCENARIO'] == scenario) &
        (energy_df['FINAL_ENERGY'] == 'ELECTRICITY')
    ]
    print(f"Scenario: {scenario}, Number of rows: {len(filtered_df)}")

filtered_df['MMBTU'].sum()

Scenario: HIGH ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT, Number of rows: 78
Scenario: MEDIUM ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT, Number of rows: 78
Scenario: REFERENCE ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT, Number of rows: 78
Scenario: LOW ELECTRICITY GROWTH - MODERATE TECHNOLOGY ADVANCEMENT, Number of rows: 83
Scenario: ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT, Number of rows: 78


In [28]:
filtered_df['MMBTU'].sum()

356800523.73745614

In [33]:
us_states['NAME'].unique()

array(['Mississippi', 'North Carolina', 'Oklahoma', 'Virginia',
       'West Virginia', 'Louisiana', 'Michigan', 'Massachusetts', 'Idaho',
       'Florida', 'Nebraska', 'Washington', 'New Mexico', 'Puerto Rico',
       'South Dakota', 'Texas', 'California', 'Alabama', 'Georgia',
       'Pennsylvania', 'Missouri', 'Colorado', 'Utah', 'Tennessee',
       'Wyoming', 'New York', 'Kansas', 'Alaska', 'Nevada', 'Illinois',
       'Vermont', 'Montana', 'Iowa', 'South Carolina', 'New Hampshire',
       'Arizona', 'District of Columbia', 'American Samoa',
       'United States Virgin Islands', 'New Jersey', 'Maryland', 'Maine',
       'Hawaii', 'Delaware', 'Guam',
       'Commonwealth of the Northern Mariana Islands', 'Rhode Island',
       'Kentucky', 'Ohio', 'Wisconsin', 'Oregon', 'North Dakota',
       'Arkansas', 'Indiana', 'Minnesota', 'Connecticut'], dtype=object)

``` python
us_states['NAME'].unique()
array(['Mississippi', 'North Carolina', 'Oklahoma', 'Virginia',
       'West Virginia', 'Louisiana', 'Michigan', 'Massachusetts', 'Idaho',
       'Florida', 'Nebraska', 'Washington', 'New Mexico', 'Puerto Rico',
       'South Dakota', 'Texas', 'California', 'Alabama', 'Georgia',
       'Pennsylvania', 'Missouri', 'Colorado', 'Utah', 'Tennessee',
       'Wyoming', 'New York', 'Kansas', 'Alaska', 'Nevada', 'Illinois',
       'Vermont', 'Montana', 'Iowa', 'South Carolina', 'New Hampshire',
       'Arizona', 'District of Columbia', 'American Samoa',
       'United States Virgin Islands', 'New Jersey', 'Maryland', 'Maine',
       'Hawaii', 'Delaware', 'Guam',
       'Commonwealth of the Northern Mariana Islands', 'Rhode Island',
       'Kentucky', 'Ohio', 'Wisconsin', 'Oregon', 'North Dakota',
       'Arkansas', 'Indiana', 'Minnesota', 'Connecticut'], dtype=object)

       
energy_df['STATE'].unique()
array(['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA',
       'COLORADO', 'CONNECTICUT', 'DELAWARE', 'DISTRICT OF COLUMBIA',
       'FLORIDA', 'GEORGIA', 'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA',
       'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND',
       'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI',
       'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE',
       'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA',
       'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 'OREGON', 'PENNSYLVANIA',
       'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE',
       'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON',
       'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'], dtype=object)

```

In [48]:
# Create summary dataframe for each state, scenario, sector, and year
# Filter for ELECTRICITY only
energy_electricity = energy_df[energy_df['FINAL_ENERGY'] == 'ELECTRICITY'].copy()

# Filter for valid years (2017-2050)
valid_years = list(range(2017, 2051))
energy_electricity = energy_electricity[energy_electricity['YEAR'].isin(valid_years)].copy()

# Group by STATE, SCENARIO, SECTOR, and YEAR, then sum MMBTU
energy_summary = energy_electricity.groupby(['STATE', 'SCENARIO', 'SECTOR', 'YEAR'])['MMBTU'].sum().reset_index()

# Convert STATE name to match us_states format (title case)
energy_summary['STATE_TITLE'] = energy_summary['STATE'].str.title()

# Merge with us_states geodataframe
energy_summary_gdf = pd.merge(
    energy_summary,
    us_states[['NAME', 'geometry']],
    left_on='STATE_TITLE',
    right_on='NAME',
    how='left'
)

# Convert to GeoDataFrame
energy_summary_gdf = gpd.GeoDataFrame(energy_summary_gdf, geometry='geometry', crs=us_states.crs)

# Display summary
print(f"Total rows in summary: {len(energy_summary_gdf)}")
print(f"States covered: {energy_summary_gdf['NAME'].nunique()}")
print(f"Scenarios: {energy_summary_gdf['SCENARIO'].nunique()}")
print(f"Sectors: {energy_summary_gdf['SECTOR'].nunique()}")
print(f"Years: {energy_summary_gdf['YEAR'].nunique()} ({energy_summary_gdf['YEAR'].min()}-{energy_summary_gdf['YEAR'].max()})")
print("\nFirst few rows:")
energy_summary_gdf


Total rows in summary: 34680
States covered: 50
Scenarios: 5
Sectors: 4
Years: 34 (2017-2050)

First few rows:


,STATE,SCENARIO,SECTOR,YEAR,MMBTU,STATE_TITLE,NAME,geometry
0,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,COMMERCIAL,2017,7.063012e+07,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
1,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,COMMERCIAL,2018,7.142212e+07,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
2,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,COMMERCIAL,2019,7.224714e+07,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
3,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,COMMERCIAL,2020,7.315417e+07,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
4,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,COMMERCIAL,2021,7.401815e+07,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
...,...,...,...,...,...,...,...,...
34675,WYOMING,REFERENCE ELECTRIFICATION - MODERATE TECHNOLOG...,TRANSPORTATION,2046,8.069778e+05,Wyoming,Wyoming,"POLYGON ((-111.05456 45.00096, -111.04507 45.0..."
34676,WYOMING,REFERENCE ELECTRIFICATION - MODERATE TECHNOLOG...,TRANSPORTATION,2047,8.153339e+05,Wyoming,Wyoming,"POLYGON ((-111.05456 45.00096, -111.04507 45.0..."
34677,WYOMING,REFERENCE ELECTRIFICATION - MODERATE TECHNOLOG...,TRANSPORTATION,2048,8.236091e+05,Wyoming,Wyoming,"POLYGON ((-111.05456 45.00096, -111.04507 45.0..."
34678,WYOMING,REFERENCE ELECTRIFICATION - MODERATE TECHNOLOG...,TRANSPORTATION,2049,8.318235e+05,Wyoming,Wyoming,"POLYGON ((-111.05456 45.00096, -111.04507 45.0..."


In [44]:
# Alternative: Summary by STATE, SCENARIO, and YEAR (sum across all sectors)
energy_summary_by_state_scenario_year = energy_electricity.groupby(['STATE', 'SCENARIO', 'YEAR'])['MMBTU'].sum().reset_index()
energy_summary_by_state_scenario_year['STATE_TITLE'] = energy_summary_by_state_scenario_year['STATE'].str.title()

# Merge with us_states
energy_summary_by_state_scenario_year_gdf = pd.merge(
    energy_summary_by_state_scenario_year,
    us_states[['NAME', 'geometry']],
    left_on='STATE_TITLE',
    right_on='NAME',
    how='left'
)
energy_summary_by_state_scenario_year_gdf = gpd.GeoDataFrame(
    energy_summary_by_state_scenario_year_gdf, 
    geometry='geometry', 
    crs=us_states.crs
)

print("Summary by STATE, SCENARIO, and YEAR (all sectors combined):")
print(f"Total rows: {len(energy_summary_by_state_scenario_year_gdf)}")
energy_summary_by_state_scenario_year_gdf


Summary by STATE, SCENARIO, and YEAR (all sectors combined):
Total rows: 8670


,STATE,SCENARIO,YEAR,MMBTU,STATE_TITLE,NAME,geometry
0,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,2017,2.457755e+08,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
1,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,2018,2.692848e+08,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
2,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,2019,2.920597e+08,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
3,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,2020,3.133971e+08,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
4,ALABAMA,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE...,2021,3.330328e+08,Alabama,Alabama,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
...,...,...,...,...,...,...,...
8665,WYOMING,REFERENCE ELECTRIFICATION - MODERATE TECHNOLOG...,2046,3.816260e+07,Wyoming,Wyoming,"POLYGON ((-111.05456 45.00096, -111.04507 45.0..."
8666,WYOMING,REFERENCE ELECTRIFICATION - MODERATE TECHNOLOG...,2047,3.846013e+07,Wyoming,Wyoming,"POLYGON ((-111.05456 45.00096, -111.04507 45.0..."
8667,WYOMING,REFERENCE ELECTRIFICATION - MODERATE TECHNOLOG...,2048,3.874785e+07,Wyoming,Wyoming,"POLYGON ((-111.05456 45.00096, -111.04507 45.0..."
8668,WYOMING,REFERENCE ELECTRIFICATION - MODERATE TECHNOLOG...,2049,3.905937e+07,Wyoming,Wyoming,"POLYGON ((-111.05456 45.00096, -111.04507 45.0..."


In [43]:
# Pivot table: STATE x SCENARIO x YEAR (all sectors combined)
# Option 1: STATE x SCENARIO (for a specific year, e.g., 2050)
energy_pivot_2050 = energy_summary_by_state_scenario_year[
    energy_summary_by_state_scenario_year['YEAR'] == 2050
].pivot_table(
    index='STATE',
    columns='SCENARIO',
    values='MMBTU',
    aggfunc='sum'
)

print("Pivot table: STATE (rows) x SCENARIO (columns) for YEAR 2050")
print(f"Shape: {energy_pivot_2050.shape}")
energy_pivot_2050

Pivot table: STATE (rows) x SCENARIO (columns) for YEAR 2050
Shape: (51, 5)


SCENARIO,ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT,HIGH ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT,LOW ELECTRICITY GROWTH - MODERATE TECHNOLOGY ADVANCEMENT,MEDIUM ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT,REFERENCE ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT
STATE,,,,,
ALABAMA,5.137998e+08,4.144200e+08,2.452857e+08,3.643564e+08,3.038851e+08
ALASKA,5.811656e+07,4.546795e+07,2.562125e+07,3.909029e+07,3.163331e+07
ARIZONA,4.634156e+08,3.834774e+08,2.248143e+08,3.329114e+08,2.742364e+08
ARKANSAS,3.094151e+08,2.494506e+08,1.509550e+08,2.187601e+08,1.855723e+08
CALIFORNIA,2.596958e+09,2.106597e+09,1.151782e+09,1.777577e+09,1.404504e+09
COLORADO,3.906618e+08,3.141188e+08,1.837406e+08,2.737598e+08,2.218660e+08
CONNECTICUT,2.398705e+08,1.983244e+08,1.069107e+08,1.684840e+08,1.323858e+08
DELAWARE,7.634906e+07,6.993436e+07,4.640613e+07,6.476500e+07,5.786832e+07
DISTRICT OF COLUMBIA,9.253731e+07,8.340846e+07,4.813579e+07,6.611445e+07,5.471554e+07


In [40]:
# Check for any unmatched states
unmatched = energy_summary_gdf[energy_summary_gdf['geometry'].isna()]
if len(unmatched) > 0:
    print("Warning: Some states could not be matched with geometry:")
    print(unmatched[['STATE', 'STATE_TITLE', 'NAME']].drop_duplicates())
else:
    print("✓ All states successfully matched with geometry!")


                     STATE           STATE_TITLE NAME
5440  DISTRICT OF COLUMBIA  District Of Columbia  NaN


In [41]:
# Option 2: Pivot table with YEAR as additional dimension
# STATE x (SCENARIO, YEAR) combination
energy_pivot_with_year = energy_summary_by_state_scenario_year.pivot_table(
    index='STATE',
    columns=['SCENARIO', 'YEAR'],
    values='MMBTU',
    aggfunc='sum'
)

print("Pivot table: STATE (rows) x (SCENARIO, YEAR) (columns)")
print(f"Shape: {energy_pivot_with_year.shape}")
print("\nColumn structure (first 10):")
print(energy_pivot_with_year.columns[:10])
print("\nFirst few rows (first 5 columns):")
energy_pivot_with_year.iloc[:5, :5]


Pivot table: STATE (rows) x (SCENARIO, YEAR) (columns)
Shape: (51, 170)

Column structure (first 10):
MultiIndex([('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT', ...),
            ('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT', ...),
            ('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT', ...),
            ('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT', ...),
            ('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT', ...),
            ('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT', ...),
            ('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT', ...),
            ('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT', ...),
            ('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT', ...),
            ('ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOG

SCENARIO   ELECTRIFICATION TECHNICAL POTENTIAL - MODERATE TECHNOLOGY ADVANCEMENT  \
YEAR                                                                        2017   
STATE                                                                              
ALABAMA                                          2.457755e+08                      
ALASKA                                           2.567991e+07                      
ARIZONA                                          2.277747e+08                      
ARKANSAS                                         1.416416e+08                      
CALIFORNIA                                       1.127845e+09                      

SCENARIO                                                            
YEAR                2018          2019          2020          2021  
STATE                                                               
ALABAMA     2.692848e+08  2.920597e+08  3.133971e+08  3.330328e+08  
ALASKA      2.838770e+07  3.088544e+07  3.326042e+07  3.551686e+07  
ARIZONA     2.493096e+08  2.693755e+08  2.883320e+08  3.054982e+08  
ARKANSAS    1.571960e+08  1.713963e+08  1.843846e+08  1.964132e+08  
CALIFORNIA  1.258620e+09  1.376754e+09  1.490091e+09  1.594112e+09

In [42]:
# Option 3: Pivot table: STATE x YEAR (for a specific scenario)
# Example: HIGH ELECTRIFICATION scenario
scenario_name = 'HIGH ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT'
energy_pivot_state_year = energy_summary_by_state_scenario_year[
    energy_summary_by_state_scenario_year['SCENARIO'] == scenario_name
].pivot_table(
    index='STATE',
    columns='YEAR',
    values='MMBTU',
    aggfunc='sum'
)

print(f"Pivot table: STATE (rows) x YEAR (columns) for scenario: {scenario_name}")
print(f"Shape: {energy_pivot_state_year.shape}")
energy_pivot_state_year.head(10)


Pivot table: STATE (rows) x YEAR (columns) for scenario: HIGH ELECTRIFICATION - MODERATE TECHNOLOGY ADVANCEMENT
Shape: (51, 34)


YEAR,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
STATE,,,,,,,,,,,,,,,,,,,,,
ALABAMA,2.458805e+08,2.470365e+08,2.489840e+08,2.504799e+08,2.527640e+08,2.562181e+08,2.589790e+08,2.616763e+08,2.640626e+08,2.666457e+08,...,3.591583e+08,3.656271e+08,3.720249e+08,3.782084e+08,3.842264e+08,3.903151e+08,3.962609e+08,4.022067e+08,4.082779e+08,4.144200e+08
ALASKA,2.552403e+07,2.586476e+07,2.608802e+07,2.625581e+07,2.651366e+07,2.678738e+07,2.708169e+07,2.739082e+07,2.768729e+07,2.802899e+07,...,3.886004e+07,3.961217e+07,4.037601e+07,4.111168e+07,4.182665e+07,4.254129e+07,4.322823e+07,4.396434e+07,4.471496e+07,4.546795e+07
ARIZONA,2.282095e+08,2.286816e+08,2.293307e+08,2.298398e+08,2.310085e+08,2.321603e+08,2.338067e+08,2.357348e+08,2.374762e+08,2.399469e+08,...,3.322152e+08,3.386035e+08,3.449405e+08,3.510027e+08,3.568139e+08,3.625975e+08,3.680032e+08,3.732157e+08,3.783917e+08,3.834774e+08
ARKANSAS,1.417575e+08,1.438285e+08,1.455301e+08,1.466766e+08,1.481797e+08,1.502399e+08,1.521493e+08,1.539980e+08,1.556700e+08,1.574476e+08,...,2.151052e+08,2.194772e+08,2.238010e+08,2.278596e+08,2.316857e+08,2.354469e+08,2.389938e+08,2.424640e+08,2.459602e+08,2.494506e+08
CALIFORNIA,1.126599e+09,1.140039e+09,1.148611e+09,1.157148e+09,1.168768e+09,1.180188e+09,1.192776e+09,1.206184e+09,1.218330e+09,1.233616e+09,...,1.767194e+09,1.806432e+09,1.845696e+09,1.883568e+09,1.920321e+09,1.957277e+09,1.992466e+09,2.030076e+09,2.068386e+09,2.106597e+09
COLORADO,1.752447e+08,1.770948e+08,1.789522e+08,1.805728e+08,1.826132e+08,1.846679e+08,1.870644e+08,1.895404e+08,1.917866e+08,1.944634e+08,...,2.716755e+08,2.768723e+08,2.820384e+08,2.870154e+08,2.918127e+08,2.966137e+08,3.011360e+08,3.055049e+08,3.098403e+08,3.141188e+08
CONNECTICUT,1.075146e+08,1.087527e+08,1.096084e+08,1.105575e+08,1.116684e+08,1.128062e+08,1.139411e+08,1.150959e+08,1.162042e+08,1.176088e+08,...,1.686242e+08,1.721896e+08,1.757026e+08,1.790953e+08,1.824051e+08,1.857810e+08,1.889665e+08,1.921256e+08,1.952625e+08,1.983244e+08
DELAWARE,4.615027e+07,4.669347e+07,4.723592e+07,4.764954e+07,4.820030e+07,4.883440e+07,4.945923e+07,4.995496e+07,5.035640e+07,5.083099e+07,...,6.250986e+07,6.335094e+07,6.422227e+07,6.505527e+07,6.586843e+07,6.668321e+07,6.748535e+07,6.828094e+07,6.909292e+07,6.993436e+07
DISTRICT OF COLUMBIA,4.334273e+07,4.361398e+07,4.394971e+07,4.432189e+07,4.479240e+07,4.532386e+07,4.586823e+07,4.642674e+07,4.700364e+07,4.767404e+07,...,6.920827e+07,7.092572e+07,7.262299e+07,7.426035e+07,7.585046e+07,7.740234e+07,7.893767e+07,8.044152e+07,8.192888e+07,8.340846e+07
